In [13]:
import keras
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
import pandas as pd

In [14]:
model_dir = './assets/'
tokenizer = BertTokenizer.from_pretrained(model_dir)
bert = TFBertModel.from_pretrained(model_dir)

Some layers from the model checkpoint at ./assets/ were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ./assets/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [15]:
df = pd.read_csv('./assets/dataset_sms_spam_v1.csv')
x = df['Teks'].str.replace('[^a-zA-Z\\d\\s:]', '', regex=True).to_list()
y = keras.utils.to_categorical(df['label'])

In [12]:
train_x, val_x, train_y, val_y = train_test_split(x,y,train_size=0.9,shuffle=True)

In [16]:
maxlen=50

In [27]:
X_train = tokenizer(text=train_x,add_special_tokens=True,max_length=maxlen,truncation=True,
              padding=True,return_tensors='tf',return_token_type_ids=False,verbose=True,return_attention_mask=True)
X_test = tokenizer(text=val_x,add_special_tokens=True,max_length=maxlen,truncation=True,
              padding=True,return_tensors='tf',return_token_type_ids=False,verbose=True,return_attention_mask=True)

In [45]:
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy

In [34]:
input_ids = Input(shape=(maxlen,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(maxlen,), dtype=tf.int32, name="attention_mask")

In [50]:
embeddings = bert(input_ids,attention_mask= input_mask)[0]
out = keras.layers.GlobalAveragePooling1D()(embeddings)
out = Dense(128,activation='relu')(out)
out = keras.layers.Dropout(0.2)(out)
out = Dense(32,activation='relu')(out)
y = Dense(3,activation='softmax')(out)
model = keras.Model(inputs=[input_ids,input_mask], outputs=y)

In [51]:
model.layers[2].trainable = False
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 50)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 50)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1106173   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   44         'attention_mask[0][0]']      
                             hidden_state=(None, 50, 76                                     

                                                                                                  
 dense_4 (Dense)             (None, 32)                   4128      ['dropout_161[0][0]']         
                                                                                                  
 dense_5 (Dense)             (None, 3)                    99        ['dense_4[0][0]']             
                                                                                                  
Total params: 110720003 (422.36 MB)
Trainable params: 102659 (401.01 KB)
Non-trainable params: 110617344 (421.97 MB)
__________________________________________________________________________________________________


In [53]:
optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
)
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

In [54]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

In [55]:
history = model.fit(
    x = {'input_ids': X_train['input_ids'], 'attention_mask': X_train['attention_mask']},
    y = train_y,
    validation_data=({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']},val_y),
    epochs=1,
    batch_size=32
)

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(



33/33 [==============================] - 260s 6s/step - loss: 1.2877 - balanced_accuracy: 0.3259 - val_loss: 0.9566 - val_balanced_accuracy: 0.5478
